<a href="https://colab.research.google.com/github/jacklmg75/data-extraction/blob/main/1_2_Classifica%C3%A7%C3%A3o_de_sentimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de Documentos por Sentimento

In [ ]:
%%capture
import nltk
# from nltk.classify import NaiveBayesClassifier
import csv

import random
from nltk.corpus import stopwords

from nltk.metrics import *
import pandas as pd

# tokenizar / quebrar em termos /  bag of words
from nltk.tokenize import word_tokenize
nltk.download('punkt') # faz o download do "tokenizador" que será usado em word_tokenize

# lista de stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

# Remoção de acentuação
!pip install unidecode
from unidecode import unidecode

# Stopwords
stop_words = stopwords.words('portuguese')

# Stemming
nltk.download('rslp')
from nltk.stem import RSLPStemmer
stemmer = RSLPStemmer()

# Leitura da base de dados

In [ ]:
# Fonte do dataset: https://github.com/clarissacastella/eniac2018
# Download do PDF: https://sol.sbc.org.br/index.php/eniac/article/view/4417

# Documentos positivos

pos_df = pd.read_csv('https://raw.githubusercontent.com/clarissacastella/eniac2018/master/train_EPTC_POA_v3nbal_1.data',
                     on_bad_lines='skip',
                     header=None,
                     names=["text"])
pos_df.head(2)

,text
0,Av. Borges de Medeiros liberada. Manifestantes...
1,RT @Trensurb: #TrensurbInforma Bom dia! Trens ...


In [ ]:
pos_df["sentimento"]=1
pos_df.head(2)

,text,sentimento
0,Av. Borges de Medeiros liberada. Manifestantes...,1
1,RT @Trensurb: #TrensurbInforma Bom dia! Trens ...,1


In [ ]:
# Documentos negativos

neg_df = pd.read_csv('https://raw.githubusercontent.com/clarissacastella/eniac2018/master/train_EPTC_POA_v3nbal_0.data',
                     on_bad_lines='skip',
                     header=None,
                     names=["text"])
neg_df["sentimento"]=-1
neg_df.head(2)

,text,sentimento
0,Trânsito acentuado nos dois sentidos da Av. Ca...,-1
1,Trânsito acentuado na saída da Capital pela Av...,-1


In [ ]:
# Documentos neutros

neu_df = pd.read_csv('https://raw.githubusercontent.com/clarissacastella/eniac2018/master/train_EPTC_POA_v3nbal_2.data',
                     on_bad_lines='skip',
                     header=None,
                     names=["text"])
neu_df["sentimento"]=0
neu_df.head(2)

,text,sentimento
0,Linhas de ônibus especiais são ativadas no fer...,0
1,RT @triunfoconcepa: CONCEPA informa: Não há pr...,0


In [ ]:
# Unificando o dataset

dataset = pd.concat([pos_df, neu_df, neg_df], axis=0, ignore_index=True)
dataset.head(5)

,text,sentimento
0,Av. Borges de Medeiros liberada. Manifestantes...,1
1,RT @Trensurb: #TrensurbInforma Bom dia! Trens ...,1
2,Bom dia! Trânsito fluindo bem nos dois sentido...,1
3,j02 - Trânsito fluindo bem nos dois sentidos d...,1
4,Trânsito fluindo bem na saída da Capital pela ...,1


In [ ]:
dataset.shape

(3951, 2)

In [ ]:
dataset.drop_duplicates(inplace=True)
dataset.shape

(2852, 2)

In [ ]:
# Dicionário de stopwords suplementar

url = 'https://raw.githubusercontent.com/stopwords-iso/stopwords-pt/master/stopwords-pt.txt'
stopwords_from_github = pd.read_csv(url,header=None)
stop_words = set(stop_words + stopwords_from_github[0].tolist())  # Agregando as novas stopwords à lista padrão


# Nossa função de preprocessamento

'''
Recebe um texto bruto (raw text)
Retorna o bag of words com todos os termos normalizados (clean text)
'''
def text_preprocess(text):
  tokens = word_tokenize(text)
  words = [word.lower() for word in tokens]
  words = [word for word in words if word.isalnum()]
  words = [w for w in words if not w in stop_words]
  words = [unidecode(w) for w in words]
  words = [stemmer.stem(w) for w in words]
  words = [w for w in words if len(w) >1 ]
  return(words)

## Criando os vetores com as frequências de termos

In [ ]:
documents_text = dataset['text']  # recebemos o texto
doc_text = documents_text[0] # Apenas a 1a linha (posição 0)
print(doc_text)
print(text_preprocess(doc_text))

Av. Borges de Medeiros liberada. Manifestantes seguem concentrados em frente ao Paço Municipal.
['av', 'borg', 'med', 'liber', 'manifest', 'segu', 'concentr', 'frent', 'pac', 'municip']


In [ ]:
# TF IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = dataset.copy()

# criando vetores para TODA a base
tfidf_vectorizer = TfidfVectorizer(tokenizer=text_preprocess)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

# similaridade
similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
df["similarity"] = similarity.ravel()

In [ ]:
print("Texto:",df.text[0])

df.sort_values("similarity",ascending=False).head()

Texto: Av. Borges de Medeiros liberada. Manifestantes seguem concentrados em frente ao Paço Municipal.


,text,sentimento,similarity
0,Av. Borges de Medeiros liberada. Manifestantes...,1,1.000000
2198,Manifestantes se concentram em frente ao Paço ...,-1,0.872919
1206,Grupo acessou a Av. Borges de Medeiros e se co...,0,0.829686
304,Av. Borges de Medeiros liberada,1,0.482482
1107,Após parada no Paço Municipal,0,0.449902


## Classificação

De posse dos vetores de termos, além de fazer a similaridade individual, podemos treinar um algoritmo de machine learning para aprender a classificar automaticamente o sentimento de cada documento.

## Dividindo a base de treino e teste

In [ ]:
#scikit-learn: https://scikit-learn.org/stable/install.html

from sklearn.model_selection import train_test_split

<img src="https://www.dropbox.com/s/a6cxb206ug3dtnb/split-dataset.png?dl=1" alt="Drawing" width="500"/>

In [ ]:
dataset.head()

,text,sentimento
0,Av. Borges de Medeiros liberada. Manifestantes...,1
1,RT @Trensurb: #TrensurbInforma Bom dia! Trens ...,1
2,Bom dia! Trânsito fluindo bem nos dois sentido...,1
3,j02 - Trânsito fluindo bem nos dois sentidos d...,1
4,Trânsito fluindo bem na saída da Capital pela ...,1


In [ ]:
# Split into training and testing data

X = dataset['text']
y = dataset['sentimento']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

In [ ]:
# Base de treino com os documentos

X_train.head()

217     Direto das câmeras da EPTC: trânsito fluindo c...
2571    Caminhão em pane mecânica na chegada à Capital...
1022    Previsão para estar terça ?? https://t.co/TIfs...
3525             Cabos de alta tensão estão caídos na via
851     RT @triunfoconcepa: CONCEPA informa: Içamento ...
Name: text, dtype: object

In [ ]:
# Gabarito de sentimento

y_train.head()

217     1
2571   -1
1022    0
3525   -1
851     0
Name: sentimento, dtype: int64

In [ ]:
# criando vetores exclusivamente para a base de treino

tfidf_vectorizer = TfidfVectorizer(tokenizer=text_preprocess)
X_train_matrix = tfidf_vectorizer.fit_transform(X_train)

In [ ]:
# Treinamos um modelo com base na matriz de termos e a classe de cada documento.
# https://blogdozouza.wordpress.com/2019/04/10/algoritmo-svm-maquina-de-vetores-de-suporte-a-partir-de-exemplos-e-codigo-python-e-r/

from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(X_train_matrix, y_train)

MultinomialNB()

## Testando nosso modelo treinado

In [ ]:
# Base de teste com os documentos

pd.set_option('max_colwidth', 200)

X_test.head(2)

1365                                                                        Serviços de atendimento restabelecidos: 118 e 156 voltaram a operar.
2534    Vias bloqueadas parcialmente com acúmulo de água: ; Souza Reis (Antes da Sertório) ; Ceará entre 25 de Fevereir… https://t.co/kh16OMXvmu
Name: text, dtype: object

In [ ]:
# Criamos a matriz de frequências para alguns documentos

X_test_matrix = tfidf_vectorizer.transform(X_test.head(2))

# Executamos a predição
## Observe que executamos a predição sem mostrar para o modelo qual é a classe correta.
## Utilizamos apenas os documentos como input

pred = model.predict(X_test_matrix)

print(pred)

[ 0 -1]


In [ ]:
# Conferindo as classes originais (nosso gabarito com o sentimento correto)

real = y_test.head(2).values

print(real)

[ 0 -1]


In [ ]:
# Criando os vetores para toda a base de testes

X_test_matrix = tfidf_vectorizer.transform(X_test)

In [ ]:
X_test = pd.DataFrame(X_test)

X_test["sentimento_real"] = y_test
X_test["sentimento_predito"] = model.predict(X_test_matrix)

X_test.head(10)

,text,sentimento_real,sentimento_predito
1365,Serviços de atendimento restabelecidos: 118 e 156 voltaram a operar.,0,0
2534,Vias bloqueadas parcialmente com acúmulo de água: ; Souza Reis (Antes da Sertório) ; Ceará entre 25 de Fevereir… https://t.co/kh16OMXvmu,-1,-1
1591,Direto das câmeras da EPTC: Trânsito acentuado na Av. da Legalidade e da Democracia em direção à Capital ne… https://t.co/Ji6NZvDDvE,0,-1
2976,Atropelamento por carro na Av. Mauá,-1,-1
826,Direto das câmeras da EPTC: delegação do Grêmio acessa a Av. Farrapos e se aproximam da Av. Cairú,0,-1
3718,Trânsito já começa a acentuar na saída da capital pela Av. da Legalidade e da Democracia. Av. Sertório apre… https://t.co/Hf1rBhO66A,-1,-1
968,armorel67 @TransitoPOARS @armorel67 obrigada pelo aviso,0,0
2920,Acidente entre dois carros na Av. São Pedro,-1,-1
1111,Atenção usuários! https://t.co/c2BeodtUxH,0,0
2209,Semáforo com problemas no cruzamento da Av. Prof Oscar Pereira x R. Francisco Martins.,-1,-1


In [ ]:
# Acurácia: Percentual de documentos classificados corretamente na base

model.score(X_test_matrix, y_test)

0.7784011220196353

### Testando o modelo com alguns textos livres

In [ ]:
model.predict(tfidf_vectorizer.transform(['Transito fluindo bem na capital']))[0]

1

In [ ]:
model.predict(tfidf_vectorizer.transform(['Transito pesado na capital']))[0]

-1

In [ ]:
model.predict(tfidf_vectorizer.transform(['Onibus circulam na capital']))[0]

0

In [ ]:
# Como prometido, mais um linkzin da Web aí pra falar sobre o Naive Bayes.
# https://www.organicadigital.com/blog/algoritmo-de-classificacao-naive-bayes/

# Mas é isso gente. Uma vez que a gente sabe o que a gente quer fica muito mais
# fácil encontrar as respostas e exemplos que precisamos. Stackoverflow e Google são nossos amigos :D

# El fin